In [2]:
from utils.spark_session import get_spark_session
from pyspark.sql import SparkSession

spark: SparkSession = get_spark_session()

25/10/06 10:18:14 WARN Utils: Your hostname, DESKTOP-68EC7N1 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/06 10:18:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dottier/venvs/flights_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dottier/.ivy2/cache
The jars for the packages stored in: /home/dottier/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-001b984b-9154-418a-b310-b2f83e2fbb56;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 104ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   

In [2]:
spark.sql("DROP TABLE IF EXISTS bronze.flights")

25/10/01 10:23:32 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/01 10:23:32 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/01 10:23:33 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/01 10:23:33 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/10/01 10:23:33 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [2]:
spark.sql("SHOW TABLES IN bronze").show()

25/10/03 15:07:55 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/03 15:07:55 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/03 15:07:56 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/03 15:07:56 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/10/03 15:07:57 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze|  flights|      false|
+---------+---------+-----------+



In [3]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, to_timestamp, regexp_extract, coalesce

def transform_timestamps(df: DataFrame) -> DataFrame:
    source_format = "yyyy-MM-dd HH:mm'Z'"
    iso_8601_format = "yyyy-MM-dd'T'HH:mm:ss'Z'"

    utc_cols = [c for c in df.columns if c.endswith("_utc")]
    local_cols = [c for c in df.columns if c.endswith("_local")]

    for utc_col in utc_cols:
        df = df.withColumn(
            utc_col,
            coalesce(
                to_timestamp(col(utc_col), source_format),
                to_timestamp(col(utc_col), iso_8601_format),
            )
        )

    for local_col in local_cols:
        df = df.withColumn(
            local_col + "_ntz",
            to_timestamp(
                regexp_extract(col(local_col), r"(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2})", 1)
            ).cast("timestamp_ntz")
        )

    return df

In [4]:
from utils.io_utils import read_df

bronze_flights = read_df(spark, "bronze", "flights")
bronze_airports = read_df(spark, "bronze", "airports")

25/10/06 10:18:21 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/06 10:18:21 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/06 10:18:22 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/06 10:18:22 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/10/06 10:18:22 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


Successfully read bronze.flights
Successfully read bronze.airports


In [5]:
import logging, re
import pandas as pd
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StringType

def camel_to_snake(name: str) -> str:
    """Converts a single string from camelCase to snake_case."""
    if name is None:
        return None
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()


def flatten_df(df: DataFrame, cols_to_snake_case_values: set[str] | None = None) -> DataFrame:
    if cols_to_snake_case_values is None:
        cols_to_snake_case_values = set()

    @pandas_udf(StringType())
    def camel_to_snake_vectorized(series: pd.Series) -> pd.Series:
        return series.str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True).str.lower()
    
    flat_cols = []

    def get_flat_cols(schema: StructType, prefix: str = ""):
        for field in schema.fields:
            full_col_name = f"{prefix}.{field.name}" if prefix else field.name

            if isinstance(field.dataType, StructType):
                get_flat_cols(field.dataType, prefix=f"{full_col_name}")
            else:
                alias_name = camel_to_snake(full_col_name.replace(".", "_"))

                if alias_name in cols_to_snake_case_values and isinstance(field.dataType, StringType):
                    flat_cols.append(
                        camel_to_snake_vectorized(col(full_col_name)).alias(alias_name)
                    )
                else:
                    flat_cols.append(
                        col(full_col_name).alias(alias_name)
                    )

    try:
        get_flat_cols(df.schema)
        
        if not flat_cols:
            logging.warning("flatten_df resulted in no columns. Returning an empty DataFrame.")
            return df.sparkSession.createDataFrame([], schema=StructType([]))
            
        return df.select(flat_cols)

    except Exception as e:
        logging.error("An error occurred during DataFrame flattening.", exc_info=True)
        raise

In [6]:
from pyspark.sql.functions import explode

departure_exploded_df = (
    bronze_flights.select(
        explode("departures").alias("data"),
        "airport",
        "ingestion_hour",
    )
)

arrival_exploded_df = (
    bronze_flights.select(
        explode("arrivals").alias("data"),
        "airport",
        "ingestion_hour",
    )
)

In [7]:
from pyspark.sql.functions import coalesce, concat_ws, lower, trim, lit, xxhash64, date_trunc

flattened_dep_df = flatten_df(
    departure_exploded_df.select(
        "data.*",
        "airport",
        "ingestion_hour"
    )
)
flattened_dep_df = transform_timestamps(flattened_dep_df)

enriched_dep_df = (
    flattened_dep_df
    .withColumn("departure_airport_pk", col("airport"))
    .withColumn(
        "arrival_airport_pk",
        coalesce(
            col("arrival_airport_iata"),
            col("arrival_airport_icao"),
            concat_ws(
                "|",
                lit("PSEUDO"),
                xxhash64(lower(trim(col("arrival_airport_name"))))
            )
        )
    )
    .withColumn(
        "best_scheduled_time_utc",
        coalesce(col("departure_scheduled_time_utc"), col("arrival_scheduled_time_utc"))
    )
    .withColumn(
        "flight_composite_pk",
        concat_ws(
            "|",
            col("best_scheduled_time_utc"),
            col("number"),
            col("departure_airport_pk"),
            col("arrival_airport_pk")
        )
    )
    .withColumn(
        "flight_pk",
        xxhash64(col("flight_composite_pk"))
    )
)

flattened_arr_df = flatten_df(
    arrival_exploded_df.select(
        "data.*",
        "airport",
        "ingestion_hour"
    )
)
flattened_arr_df = transform_timestamps(flattened_arr_df)

enriched_arr_df = (
    flattened_arr_df
    .withColumn(
        "departure_airport_pk",
        coalesce(
            col("departure_airport_iata"),
            col("departure_airport_icao"),
            concat_ws(
                "|",
                lit("PSEUDO"),
                xxhash64(lower(trim(col("departure_airport_name"))))
            )
        )
    )
    .withColumn("arrival_airport_pk", col("airport"))
    .withColumn(
        "best_scheduled_time_utc",
        coalesce(col("departure_scheduled_time_utc"), col("arrival_scheduled_time_utc"))
    )
    .withColumn(
        "flight_composite_pk",
        concat_ws(
            "|",
            col("best_scheduled_time_utc"),
            col("number"),
            col("departure_airport_pk"),
            col("arrival_airport_pk")
        )
    )
    .withColumn(
        "flight_pk",
        xxhash64(col("flight_composite_pk"))
    )
)

In [8]:
flattened_dep_df.show(5)

25/10/03 15:08:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [48]:
import importlib
import utils.expression_utils  # make sure the module is imported
importlib.reload(utils.expression_utils)

from utils.expression_utils import get_select_expressions


In [8]:
from utils.expression_utils import get_select_expressions
from pyspark.sql.functions import when, length

union_flights = enriched_dep_df.union(enriched_arr_df).dropDuplicates(["flight_pk"])

# Airline iata codes are pretty inconsistent
enriched_union_flights = (
    union_flights
    .withColumn(
        "airline_pk",
        coalesce(
            col("airline_iata"),
            col("airline_icao"),
            when(
                (length(col("airline_name")).between(3, 4)) & 
                (col("airline_name").rlike("^[A-Z]+$")),
                col("airline_name")             # treat as code
            ).otherwise(
                concat_ws(
                    "|",
                    lit("PSEUDO"),
                    xxhash64(lower(trim(col("airline_name"))))
                )
            )
        )
    )

    .withColumn(
        "aircraft_pk",
        when(
            col("aircraft_reg").isNull() & 
            col("aircraft_mode_s").isNull() & 
            col("aircraft_model").isNull(),
            lit("UNKNOWN")
        ).otherwise(
            coalesce(
                col("aircraft_reg"),
                col("aircraft_mode_s"),
                col("aircraft_model"),
            )
        )
    ).withColumn(
        "aircraft_pk_type",
        when(col("aircraft_reg").isNotNull(), lit("reg"))
        .when(col("aircraft_mode_s").isNotNull(), lit("mode_s"))
        .when(col("aircraft_model").isNotNull(), lit("model"))
        .otherwise(lit("unknown"))
    )

    .withColumn(
        "dep_local_timezone",
        regexp_extract(col("departure_scheduled_time_local"), r"([+-]\d{2}:\d{2}|Z)$", 1)
    ).withColumn(
        "arr_local_timezone",
        regexp_extract(col("arrival_scheduled_time_local"), r"([+-]\d{2}:\d{2}|Z)$", 1)
    )
)

flights_select_exprs = get_select_expressions("silver", "fct_flights")

fct_flights = enriched_union_flights.select(*flights_select_exprs)

In [8]:
print(fct_flights.count())
a = fct_flights.dropDuplicates(['flight_pk'])
print(a.count())

25/10/06 08:50:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


101104
101104


In [24]:
fct_flights.printSchema()

root
 |-- flight_pk: long (nullable = false)
 |-- flight_composite_pk: string (nullable = false)
 |-- departure_airport_pk: string (nullable = true)
 |-- arrival_airport_pk: string (nullable = true)
 |-- airline_pk: string (nullable = true)
 |-- aircraft_pk: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_predicted_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- departure_scheduled_time_local_ntz: timestamp_ntz (nullable = true)
 |-- departure_revised_time_local_ntz: timestamp_ntz (nullable = true)
 |-- departure_predicted_time_local_ntz: timestamp_ntz (nullable = true)
 |-- departure_runway_time_local_ntz: timestamp_ntz (nulla

In [13]:
fct_flights.show(30)

+--------------------+--------------------+--------------------+------------------+----------+--------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+--------------------+-------------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+-------------------+--------------------+-------------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-------------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|           flight_pk| flight_composite_pk|departure_airport_pk|arrival_airport_pk|airline_pk|         aircraft_pk|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_at_utc| dep_revised

In [9]:
enriched_union_flights.show(5)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [28]:
col_names = []
for c in fct_flights.columns:
    col_names.append(c)

from pyspark.sql import functions as sf

fct_flights.select([
    sf.count(sf.col(c)).alias(c) for c in col_names
]).show()

+---------+-------------------+--------------------+------------------+----------+------------+-------------+---------------+-------------+----------------+--------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-----------+-----------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_pk|flight_composite_pk|departure_airport_pk|arrival_airport_pk|airline_pk|aircraft_reg|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_at_utc|dep_revised_at_utc|dep_predicted_at_utc|dep_runway_at_utc|dep_

In [17]:
# fct_flights.filter(col("dep_runway_at_utc").isNull() & (col("ingestion_hour") != "2025-10-02-12")).show(10)
fct_flights.filter(col("departure_scheduled_time_utc").isNull() & col("arrival_scheduled_time_utc").isNull()).show(10)

+---------+-------------------+--------------------+------------------+----------+------------+-------------+---------------+-------------+----------------+--------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-----------+-----------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_pk|flight_composite_pk|departure_airport_pk|arrival_airport_pk|airline_pk|aircraft_reg|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_at_utc|dep_revised_at_utc|dep_predicted_at_utc|dep_runway_at_utc|dep_

In [29]:
from pyspark.sql.functions import col, count, desc, asc

freq = (
    fct_flights.groupBy("flight_pk")
    .agg(count("*").alias("count"))
    .orderBy(desc("count"))
)

freq.show()

+--------------------+-----+
|           flight_pk|count|
+--------------------+-----+
| 3102861892797399091|    1|
| 9193410308441172342|    1|
| 7346213422395906368|    1|
| 5649166409100616710|    1|
| 2238441278163207745|    1|
|-4194597709196844967|    1|
| 1238321533318848982|    1|
|-7775228834753590174|    1|
|-8016415414734154826|    1|
| 9160834239411724389|    1|
| 2751345054993881973|    1|
|-1045403711189077840|    1|
| -864599073279825470|    1|
|-6257820157337637221|    1|
| 8545835900381982364|    1|
| -660628759902659641|    1|
|  220370909365751531|    1|
|-7018307032246903774|    1|
|-8846759895064165749|    1|
| 3905420773075731566|    1|
+--------------------+-----+
only showing top 20 rows



In [35]:
top_flight_number = freq.collect()[0]["flight_pk"]

all_records_top_flight = fct_flights.filter(col("flight_pk") == -5535522808985056397)

all_records_top_flight.show(20, truncate=False)

+--------------------+-------------------------------+--------------------+------------------+----------+------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+-------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-------------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_pk           |flight_composite_pk            |departure_airport_pk|arrival_airport_pk|airline_pk|aircraft_reg|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_at_utc|dep_revise

In [10]:
fct_flights.count()

120593

In [50]:
from pyspark.sql.functions import col

top30_codes = [
    "LHR","IST","CDG","AMS","MAD","FRA","BCN","FCO","SVO","LGW",
    "MUC","LIS","DUB","PMI","ORY","ATH","VIE","ZRH","MAN","CPH",
    "STN","MXP","OSL","BER","AGP","BRU","ARN","WAW","LED","DUS"
]


check = (
    fct_flights
    .select("flight_pk", "departure_airport_pk", "arrival_airport_pk")
    .distinct()
    .withColumn("dep_in_top30", col("departure_airport_pk").isin(top30_codes))
    .withColumn("arr_in_top30", col("arrival_airport_pk").isin(top30_codes))
)

check.filter(col("dep_in_top30") & col("arr_in_top30")).count()


21223

In [15]:
col_names = []
for c in flattened_dep_df.columns:
    col_names.append(c)

from pyspark.sql import functions as sf

flattened_dep_df.select([
    sf.count(sf.col(c)).alias(c) for c in col_names
]).show()

flattened_dep_df.show(1)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [54]:
flattened_dep_df.show(5)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [10]:
# bronze_airports.show(5)

flattened_airports_df = flatten_df(bronze_airports)

detailed_airports_select_exprs = get_select_expressions("silver", "dim_airports_detailed")

dim_airports_detailed = flattened_airports_df.select(*detailed_airports_select_exprs)
dim_airports_detailed.show()


+----+----+--------------------+--------------------+-----------------+---------+---------+--------------+------------+--------------+--------------+--------------+-----------------+
|iata|icao|          short_name|           full_name|municipality_name| latitude|longitude|elevation_feet|country_code|  country_name|continent_code|continent_name|        time_zone|
+----+----+--------------------+--------------------+-----------------+---------+---------+--------------+------------+--------------+--------------+--------------+-----------------+
| AMS|EHAM|            Schiphol|  Amsterdam Schiphol|        Amsterdam|  52.3086| 4.763889|         -11.0|          NL|   Netherlands|            EU|        Europe| Europe/Amsterdam|
| MAD|LEMD|Adolfo Suárez –Ba...|Madrid Adolfo Suá...|           Madrid|  40.4936| -3.56676|        1998.0|          ES|         Spain|            EU|        Europe|    Europe/Madrid|
| CDG|LFPG|   Charles de Gaulle|Paris Charles de ...|            Paris|  49.0128| 2.5

In [13]:
test_cols = []
for c in test.columns:
    test_cols.append(c)

import pyspark.sql.functions as sf

test.select([
    sf.count(sf.col(c)).alias(c) for c in test_cols
]).show()

+----+----+----------+----------+---------+-----------------+------------+------------+---------------+------------+--------------+------------+--------------+------------+------------+--------------+--------------+---------+-------------+--------------+------------+-------------+-----------------+----------------+-------+---------------------+-----------------------+-------------------------+-------+---------------------------+
|icao|iata|local_code|short_name|full_name|municipality_name|location_lat|location_lon|elevation_meter|elevation_km|elevation_mile|elevation_nm|elevation_feet|country_code|country_name|continent_code|continent_name|time_zone|urls_web_site|urls_wikipedia|urls_twitter|urls_live_atc|urls_flight_radar|urls_google_maps|runways|current_time_time_utc|current_time_time_local|current_time_time_zone_id|airport|current_time_time_local_ntz|
+----+----+----------+----------+---------+-----------------+------------+------------+---------------+------------+--------------+---

In [11]:
arr_airports = (
    enriched_dep_df.select(
        col("arrival_airport_pk").alias("airport_pk"),
        col("arrival_airport_iata").alias("airport_iata"),
        col("arrival_airport_icao").alias("airport_icao"),
        col("arrival_airport_name").alias("airport_name"),
        col("arrival_airport_time_zone").alias("airport_time_zone"),
    )
)

dep_airports = (
    enriched_arr_df.select(
        col("departure_airport_pk").alias("airport_pk"),
        col("departure_airport_iata").alias("airport_iata"),
        col("departure_airport_icao").alias("airport_icao"),
        col("departure_airport_name").alias("airport_name"),
        col("departure_airport_time_zone").alias("airport_time_zone"),
    )
)

dim_airports_basic = dep_airports.union(arr_airports).dropDuplicates(["airport_pk"])
print(dim_airports_basic.count())

dim_airports_basic.filter(col("airport_name") == "London").show(20, truncate=False)
# dim_airports_basic.filter(col("airport_iata").isNull()).show(200, truncate=False)
dim_airports_basic.filter(col("airport_iata").isNotNull()).count()

# dim_airports_basic.write.mode("overwrite").csv("/home/dottier/flights_etl/data/airports.csv", header=True)

1142


+---------------------------+------------+------------+------------+-----------------+
|airport_pk                 |airport_iata|airport_icao|airport_name|airport_time_zone|
+---------------------------+------------+------------+------------+-----------------+
|BQH                        |BQH         |EGKB        |London      |Europe/London    |
|LCY                        |LCY         |EGLC        |London      |Europe/London    |
|LGW                        |LGW         |EGKK        |London      |Europe/London    |
|LHR                        |LHR         |EGLL        |London      |Europe/London    |
|LTN                        |LTN         |EGGW        |London      |Europe/London    |
|NHT                        |NHT         |EGWU        |London      |Europe/London    |
|PSEUDO|-5091017362067582158|NULL        |NULL        |London      |NULL             |
|STN                        |STN         |EGSS        |London      |Europe/London    |
+---------------------------+------------+-

880

In [12]:
from pyspark.sql.functions import col, coalesce, xxhash64

# Perform a left join to combine the two datasets on the IATA code.
enriched_df = dim_airports_basic.alias("basic").join(
    dim_airports_detailed.alias("detailed"),
    on=col("basic.airport_iata") == col("detailed.iata"), # Assuming 'iata' is the key in detailed
    how="left"
)

enriched_df.printSchema()

# 2. Define the final, authoritative dimension table using the "Coalesce and Select" pattern.
#    This is the most important step.
dim_airports_final = enriched_df.select(
    # --- Keys and Identifiers ---
    # The PK comes from the basic table, ensuring all airports have one.
    col("basic.airport_pk"),
    col("basic.airport_iata"),
    coalesce(col("detailed.icao"), col("basic.airport_icao")).alias("airport_icao"),

    # --- Naming and Location ---
    # Prefer the detailed, full name, but fall back to the basic name.
    coalesce(col("detailed.full_name"), col("basic.airport_name")).alias("airport_name"),
    col("detailed.municipality_name"), # This only exists in the detailed source
    col("detailed.country_name"),
    col("detailed.continent_name"),
    
    # --- Technical Specs ---
    # The detailed source is the only one with this info. Nulls are acceptable.
    col("detailed.latitude"),
    col("detailed.longitude"),
    col("detailed.elevation_feet"),
    
    # --- Timezone ---
    # Prefer the detailed timezone, but fall back to the basic one.
    coalesce(col("detailed.time_zone"), col("basic.airport_time_zone")).alias("airport_time_zone")
)

# Now, dim_airports_final is a single, clean, authoritative table.
dim_airports_final.show()

root
 |-- airport_pk: string (nullable = false)
 |-- airport_iata: string (nullable = true)
 |-- airport_icao: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- airport_time_zone: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- municipality_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation_feet: double (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- continent_code: string (nullable = true)
 |-- continent_name: string (nullable = true)
 |-- time_zone: string (nullable = true)

+----------+------------+------------+----------------+-----------------+------------+--------------+--------+---------+--------------+------------------+
|airport_pk|airport_iata|airport_icao|    airport_name|municipa

dim_runways

In [39]:
r = flattened_airports_df.select("runways")
r.printSchema()

root
 |-- runways: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- trueHdg: double (nullable = true)
 |    |    |-- length: struct (nullable = true)
 |    |    |    |-- meter: double (nullable = true)
 |    |    |    |-- km: double (nullable = true)
 |    |    |    |-- mile: double (nullable = true)
 |    |    |    |-- nm: double (nullable = true)
 |    |    |    |-- feet: double (nullable = true)
 |    |    |-- width: struct (nullable = true)
 |    |    |    |-- meter: double (nullable = true)
 |    |    |    |-- km: double (nullable = true)
 |    |    |    |-- mile: double (nullable = true)
 |    |    |    |-- nm: double (nullable = true)
 |    |    |    |-- feet: double (nullable = true)
 |    |    |-- isClosed: boolean (nullable = true)
 |    |    |-- location: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lon: double (nullable = true)
 |    |    |--

In [13]:
exploded_runways = (
    flattened_airports_df.select(
        col("iata").alias("airport_pk"),
        explode("runways").alias("data")
    )
)

flattened_runways_df = flatten_df(
    exploded_runways.select(
        "airport_pk",
        "data.*"
    ))


r_select_exprs = get_select_expressions("silver", "dim_runways")
dim_runways = flattened_runways_df.select(*r_select_exprs)
dim_runways.show()
dim_runways.count()

+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+----------+
|airport_pk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet| latitude| longitude|
+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+----------+
|       AMS|         04|        41.1|Asphalt|        true|    false|     6634.0|     148.0|                     0.0|52.300358|  4.783485|
|       AMS|         06|        57.9|Asphalt|        true|    false|    11283.0|     148.0|                  820.21| 52.28792| 4.7341533|
|       AMS|         09|        86.8|Asphalt|        true|    false|    11306.0|     148.0|                  291.99|52.316628| 4.7463284|
|       AMS|        18C|       183.2|Asphalt|        true|    false|    10833.0|     148.0|                     0.0|52.331394|  4.740041|
|       AMS|        18L|       183

162

In [ ]:
dim_airports.count()
dim_airports.filter(col("iata") == "LHR").show(5)

+----+----+----------+----------+---------------+-----------------+------------+------------+---------------+------------+--------------+------------+--------------+------------+--------------+--------------+--------------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+---------------------+-----------------------+-------------------------+-------+----------+------------+------------+------------+-----------------+
|icao|iata|local_code|short_name|      full_name|municipality_name|location_lat|location_lon|elevation_meter|elevation_km|elevation_mile|elevation_nm|elevation_feet|country_code|  country_name|continent_code|continent_name|    time_zone|       urls_web_site|      urls_wikipedia|        urls_twitter|urls_live_atc|   urls_flight_radar|    urls_google_maps|             runways|current_time_time_utc|current_time_time_local|current_time_time_zone_id|airport|airport_pk|airpor

In [ ]:
dim_airlines = enriched_union_flights.select(
    col("airline_pk").alias("airline_pk"),
    col("airline_iata").alias("airline_iata"),
    col("airline_icao").alias("airline_icao"),
    col("airline_name").alias("airline_name"),
).dropDuplicates(["airline_pk"])

dim_airlines.filter(length("airline_name").between(3, 4) & (col("airline_pk") == col("airline_name"))).show(200, truncate=False)
# dim_airlines.orderBy(length("airline_pk").desc()).show(20, truncate=False)

+----------+------------+------------+------------+
|airline_pk|airline_iata|airline_icao|airline_name|
+----------+------------+------------+------------+
|AEZ       |NULL        |NULL        |AEZ         |
|AWG       |NULL        |NULL        |AWG         |
|BFX       |NULL        |NULL        |BFX         |
|BIX       |NULL        |NULL        |BIX         |
|BNOG      |NULL        |NULL        |BNOG        |
|CAT       |NULL        |NULL        |CAT         |
|CMA       |NULL        |NULL        |CMA         |
|CMS       |NULL        |NULL        |CMS         |
|CSG       |NULL        |NULL        |CSG         |
|CYP       |NULL        |NULL        |CYP         |
|DGJ       |NULL        |NULL        |DGJ         |
|DMS       |NULL        |NULL        |DMS         |
|DWW       |NULL        |NULL        |DWW         |
|EAI       |NULL        |NULL        |EAI         |
|FJO       |NULL        |NULL        |FJO         |
|FRO       |NULL        |NULL        |FRO         |
|FSF       |

In [ ]:
from pyspark.sql.functions import countDistinct, col
from pyspark.sql.functions import when, length

union_flights = enriched_dep_df.union(enriched_arr_df)

(
    dim_airports
    .groupBy("airport_name")
    .agg(
        countDistinct("airport_iata").alias("distinct_iata"),
        countDistinct("airport_icao").alias("distinct_icao"),
        countDistinct("airport_time_zone").alias("distinct_tz"),
    )
    # .filter(
    #     (col("distinct_iata") >= 1) | (col("distinct_icao") >= 1) | (col("distinct_tz") >= 1)
    # )
    .orderBy(col("distinct_iata").asc())
    .show(truncate=False)
)


+--------------------+-------------+-------------+-----------+
|airport_name        |distinct_iata|distinct_icao|distinct_tz|
+--------------------+-------------+-------------+-----------+
|Paros               |0            |0            |0          |
|Sint-Marten         |0            |0            |0          |
|Kos                 |0            |0            |0          |
|Frankfurt Hahn      |0            |0            |0          |
|Kluzh               |0            |0            |0          |
|Chisinau            |0            |0            |0          |
|Venice, Marco Polo  |0            |0            |0          |
|Speyer              |0            |0            |0          |
|Villafranca Airport |0            |0            |0          |
|Jersey              |0            |0            |0          |
|Paris, Ch. de Gaulle|0            |0            |0          |
|Birrfeld            |0            |0            |0          |
|Mitilini            |0            |0            |0    

In [42]:
dim_aircraft = enriched_union_flights.select(
    "aircraft_pk",
    "aircraft_reg",
    "aircraft_mode_s",
    "aircraft_model",
    "aircraft_pk_type"
).dropDuplicates(["aircraft_pk"])

dim_aircraft.filter(col("aircraft_pk_type") == "model").show(100)
dim_aircraft.count()

+--------------------+------------+---------------+--------------------+----------------+
|         aircraft_pk|aircraft_reg|aircraft_mode_s|      aircraft_model|aircraft_pk_type|
+--------------------+------------+---------------+--------------------+----------------+
|          ATR 42-300|        NULL|           NULL|          ATR 42-300|           model|
|          ATR 42-500|        NULL|           NULL|          ATR 42-500|           model|
| ATR 42/72 Freighter|        NULL|           NULL| ATR 42/72 Freighter|           model|
|              ATR 72|        NULL|           NULL|              ATR 72|           model|
|          ATR 72-500|        NULL|           NULL|          ATR 72-500|           model|
|          ATR 72-600|        NULL|           NULL|          ATR 72-600|           model|
|AgustaWestland AW139|        NULL|           NULL|AgustaWestland AW139|           model|
|Airbus A-320-200 ...|        NULL|           NULL|Airbus A-320-200 ...|           model|
|     Airb

7893

In [38]:
from pyspark.sql.functions import col

null_aircraft = enriched_union_flights.filter(
    col("aircraft_reg").isNull() &
    col("aircraft_mode_s").isNull() &
    col("aircraft_model").isNotNull()
)

null_aircraft.show(truncate=False)
null_aircraft.count()


+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

47106

In [ ]:
from pyspark.sql.functions import col

# df is your original DataFrame
df1 = fct_flights.alias("df1")
df2 = fct_flights.alias("df2")

df = df1.join(
    df2,
    on=(
        df1["departure_airport_iata"] == df2["arrival_airport_iata"] &
        df1["arrival_airport_iata"] == df2["departure_airport_iata"] &
        df1[]
    )
)

In [11]:
fct_flights.filter(col("airline_icao").isNull() & col("airline_iata").isNull()).show(5)

+--------------------+--------------------+----------------------+--------------------+------------+------------+------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+--------------------+-------------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-------------+-----------+-----------------------------+---------+---------+-----------+----------------+--------------+--------------+
|           flight_pk| flight_composite_pk|departure_airport_iata|arrival_airport_iata|airline_iata|airline_icao|aircraft_reg|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_a

In [20]:
import pyspark.sql.functions as sf

fct_flights.filter(sf.length(col("airline_pk")) > 4).show(5, truncate=False)

+--------------------+------------------------------------------------------------+----------------------+---------------------------+---------------------------+------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+--------------------+-------------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-------------+-----------+-----------------------------+---------+---------+-----------+----------------+--------------+--------------+
|flight_pk           |flight_composite_pk                                         |departure_airport_iata|arrival_airport_iata       |airline_pk                 |

In [ ]:
from process.utils import write_delta_table

spark.sql("CREATE DATABASE IF NOT EXISTS silver")
spark.sql("SHOW TABLES IN silver").show()

write_delta_table(fct_flights, "silver", "fct_flights", partition_cols=["ingestion_hour"])
write_delta_table(dim_airports_final, "silver", "dim_airports", write_mode="merge", primary_keys=["airport_pk"])
write_delta_table(dim_runways, "silver", "dim_runways", write_mode="merge", primary_keys=["airport_pk", "surface", "length_ft", "width_ft"])
write_delta_table(dim_airlines, "silver", "dim_airlines", write_mode="merge", primary_keys=["airline_pk"])
write_delta_table(dim_aircraft, "silver", "dim_aircraft", write_mode="merge", primary_keys=["aircraft_pk"])



NameError: name 'spark' is not defined

In [25]:
fct_flights.filter(sf.length(col("arrival_airport_iata")) > 3).select(sf.count_distinct(col("arrival_airport_iata"))).show()
# fct_flights.filter(sf.length(col("arrival_airport_iata")) > 3).show()

+------------------------------------+
|count(DISTINCT arrival_airport_iata)|
+------------------------------------+
|                                  31|
+------------------------------------+



In [10]:
departure_flights.printSchema()

root
 |-- flight_pk: string (nullable = true)
 |-- flight_composite_pk: string (nullable = false)
 |-- departure_airport_iata: string (nullable = true)
 |-- arrival_airport_iata: string (nullable = true)
 |-- airline_iata: string (nullable = true)
 |-- aircraft_reg: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_predicted_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- dep_scheduled_at_local_ntz: string (nullable = true)
 |-- dep_revised_at_local_ntz: string (nullable = true)
 |-- dep_predicted_at_local_ntz: string (nullable = true)
 |-- dep_runway_at_local_ntz: string (nullable = true)
 |-- dep_local_timezone: string (nullabl

In [26]:
from pyspark.sql.functions import explode

arrival_exploded_df = (
    bronze_flights.select(
        explode("arrivals").alias("data"),
        "airport",
        "ingestion_hour",
    )
)

arrival_exploded_df.count()

27350